In [119]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from pandas.io.parsers import read_csv

In [120]:
def lee_csv(filename):
    valores = read_csv(filename, header= None).values
    return valores.astype(float)

In [121]:
def funcion_ec_normal(X, Y):
    return np.linalg.pinv(X.transpose * X) * X.transpose * Y  #X NO se normaliza

In [122]:
def normalizar_matriz(X):
    #X_norm = [][] # tiene las mismas dimensiones que X, con los valores normalizados
    mu = np.mean(X, axis = 1) # medias de cada atributo
    sigma = np.std(X, axis = 1) # desviación estándar de cada atributo
    X_norm = []
    X_norm.append(X[0])
    for j in range(1, len(X[1])):
        X_norm.append((X[j] -  mu[j]) / sigma[j])
        
    # El valor de cada atributo ahora será el cociente entre su diferencia con la media y la desviación estándar
    # de ese atributo en los ejemplos de entrenamiento
    return np.array(X_norm), mu, sigma

In [123]:
def hipotesis(x, THETA):
    return THETA.transpose()*x # Se devuelve (la matriz traspuesta de THETA)*x

In [124]:
def funcion_coste_vectorizada(X, Y, THETA, num_muestras):
    normal = (X*THETA - Y)
    tras = normal.transpose()
    
    return (1 / 2 * num_muestras) * tras * (X*THETA - Y)

In [125]:
def descenso_gradiente(X, Y, THETA, num_muestras, alpha = 0.01, iteraciones = 1500):
    sumatorio = 0.0 #Sumatorio desde 1 hasta m        
    for j in range(0, iteraciones):
        for aux in range(0, len(THETA)):
            for i in range(num_muestras):
                sumtemp = sumatorio + ((hipotesis(X[i], THETA) - Y[i])*X[i][aux])
                sumatorio = sumtemp
            temp = THETA[aux] - alpha*(1/num_muestras)*sumatorio
            THETA[aux] = temp
            sumtemp = sumatorio = 0.0
            
    dibuja_hipotesis(X, Y, THETA)
    return THETA

In [127]:
datos_Fichero = lee_csv('ex1data2.csv')
filas = list(datos_Fichero)
num_muestras = len(filas)
columnas = 2 # De 0 a 2 (?)

vY = np.array([0 for i in range(num_muestras)])
vX = np.array([[0 for i in range (len(filas[0]))] for i in range (num_muestras)])

for x in range(len(filas)):
    for y in range (len(filas[0])+1):
        if(y == 0):
            vX [x][y] = 1
        elif(y < len(filas[0])):
            vX [x][y] = filas[x][y-1]
        else:
            vY[x] = filas[x][y-1]
            
vX_transpuesta = vX.transpose()
vX_norm, vX_media, vX_desv_std = normalizar_matriz(vX)
vY_media = np.mean(vY)
vY_desv_std = np.std(vY)
vX_norm_transpuesta = vX_norm.transpose()

THETA = np.array([0 for i in range(len(vX_norm))])

THETA = descenso_gradiente(vX_norm, vY_norm, THETA, num_muestras)

theta = descenso_gradiente(vX, vY, theta, num_muestras)
valor_normal = funcion_ec_normal(vX, vY)

NameError: name 'vY_norm' is not defined